In [1]:
import pandas as pd
from datetime import datetime, timedelta
from analyzer.live_entry_strats import LiveEntryStrats as les
from analyzer.live_exit_strats import LiveExitStrats as lxs
from time import sleep
from database.comet import Comet
from coinbase.coinbase import Coinbase as cbs
from processor.processor import Processor as p
import pytz

In [2]:
comet = Comet()

In [3]:

whitelist_symbols = [ 
                        'BTC'
                        , 'ADA'
                        , 'DOGE'
                        , 'ETH'
                        , 'SHIB'
                        , 'WLUNA'
                        ,'AVAX'
                        , 'LTC'
                        , 'DOT'
                        ,'MATIC'
                    ]
live = True
sleep_time = 3600
minimum_funds = 50

In [4]:
comet.cloud_connect()
status = "initial_load"
try:
    trading_params = comet.retrieve("cloud_trading_params")
    retrack_days = int(trading_params["retrack_days"].item())
    req = trading_params["req"].item()
    signal = trading_params["signal"].item()
    value = trading_params["value"].item()
    conservative = trading_params["conservative"].item()
    entry_strategy = trading_params["entry_strategy"].item()
    exit_strategy = trading_params["exit_strategy"].item()
    fee = 0.005
    minimum_rows = int(retrack_days * 3)
    end = datetime.now().astimezone(pytz.UTC)
    start = (end - timedelta(days=30)).astimezone(pytz.UTC)
    accounts = cbs.get_accounts()
    balance = accounts[accounts["currency"]=="USD"]["balance"].iloc[0]
    pending_orders = cbs.get_orders()
    pending_orders = p.live_column_date_processing(pending_orders.rename(columns={"created_at":"date"}))
    spots = []
    historicals = []
    status = "spots"
    for currency in whitelist_symbols:
        try:
            spot = cbs.get_current_price(currency)
            historical = cbs.get_timeframe_prices(currency,start,end,minimum_rows)
            spot["crypto"] = currency
            if "message" not in list(spot.keys()):
                spots.append(spot)
            historicals.append(historical)
        except Exception as e:
            error_message = {"date":datetime.now(),"message":str(e),"currency":currency}
#                 comet.store("cloud_errors",pd.DataFrame([error_message]))
            continue
    current_spots = pd.DataFrame(spots)
    current_historicals = pd.concat(historicals)
    current_historicals.sort_values("date",inplace=True)
    ns = []
    status = "calcs"
    for crypto in whitelist_symbols:
        try:
            crypto_sim = current_historicals[current_historicals["crypto"]==crypto].copy()
            crypto_sim.sort_values("date",inplace=True)
            crypto_sim["signal"] = crypto_sim["close"].pct_change(retrack_days)
            crypto_sim["velocity"] = crypto_sim["signal"].pct_change()
            crypto_sim["inflection"] = crypto_sim["velocity"].pct_change()
            crypto_sim["p_sign_change"] = [row[1]["velocity"] * row[1]["inflection"] < 0 for row in crypto_sim.iterrows()]
            ns.append(crypto_sim.iloc[-1])
        except Exception as e:
            error_message = {"date":datetime.now(),"message":str(e),"currency":currency}
#                 comet.store("cloud_errors",pd.DataFrame([error_message]))
            continue
    final = pd.DataFrame(ns)
    merged = final.merge(current_spots.drop("volume",axis=1),on="crypto")
    merged["ask"] = [float(x) for x in merged["ask"]]
    merged["bid"] = [float(x) for x in merged["bid"]]
    merged["price"] = [float(x) for x in merged["price"]]
#         comet.store("cloud_coinbase_hourly",merged)
    fls = []
    status = "fills"
    for currency in accounts["currency"].unique():
        fill = cbs.get_fill(currency)
        if len(fill) > 0:
            try:
                f = pd.DataFrame(fill)
                fls.append(f)
            except:
                continue
    ## sells
    if len(fls) > 0:
        fills = pd.concat(fls)
        #store_non_existing_executed_buy
        existing_fills = comet.retrieve_fills()
        if existing_fills.index.size > 1:
            existing_order_ids = list(existing_fills["order_id"])
        else:
            existing_order_ids = []
        new_fills = fills[~fills["order_id"].isin(existing_order_ids)]
        status = "sells"
        if new_fills.index.size > 1:
            incomplete_trades = new_fills[(new_fills["side"]=="buy")]
            incomplete_sells = new_fills[(new_fills["side"]=="sell")]
            incomplete_trades["size"] = [float(x) for x in incomplete_trades["size"]]
            incomplete_trades["price"] = [float(x) for x in incomplete_trades["price"]]
            for oi in incomplete_trades["order_id"].unique():
                order_trades = incomplete_trades[incomplete_trades["order_id"]==oi]
                if len([x for x in order_trades["settled"] if x == False]) == 0 and order_trades.index.size > 0:
#                         comet.store("cloud_fills",order_trades)
                    incomplete_trade = lxs.exit_analysis(exit_strategy,merged,order_trades,retrack_days,req)
                    if "sell_price" in incomplete_trade.keys():
                        sell_statement = cbs.place_sell(incomplete_trade["product_id"]
                                                        ,incomplete_trade["sell_price"]
                                                        ,incomplete_trade["size"])
#                             comet.store("cloud_orders",pd.DataFrame([sell_statement]))
                        incomplete_trade["sell_id"] = sell_statement["id"]
#                             comet.store("cloud_incomplete_trades",pd.DataFrame([incomplete_trade]))
            status = "trade_completes"
            completed_trades = new_fills[(new_fills["side"]=="sell")]
            for soi in completed_trades["order_id"].unique():
                sell_order_trades = completed_trades[completed_trades["order_id"]==soi]
                if len([x for x in sell_order_trades["settled"] if x == False]) == 0:
#                         comet.store("cloud_fills",sell_order_trades)
                    complete_trade = sell_order_trades.iloc[0]
                    order_id = complete_trade["order_id"]
                    one_half = comet.retrieve_incomplete_trade(order_id)
                    one_half["sell_date"] = complete_trade["created_at"]
                    one_half["sell_price"] = complete_trade["price"]
#                         comet.store("cloud_complete_trades",one_half)
    # ##buys
    status = "buys"
    if balance > minimum_funds:
        offerings = les.entry_analysis(entry_strategy,merged,signal,value,conservative)
        if offerings.index.size > 0:
            trade = offerings.iloc[0]
            buy_price = float(trade["price"])
            symbol = trade["crypto"]
            size = round(float(balance/(buy_price*(1+fee))),6)
            buy = cbs.place_buy(symbol,buy_price,size)
            if "message" not in buy.keys():
                print(buy)
#                     comet.store("cloud_orders",pd.DataFrame([buy]))
            else:
                buy["date"] = datetime.now()
                buy["crypto"] = symbol
                buy["size"] = size
                buy["buy_price"] = buy_price
                buy["balance"] = balance
#                     comet.store("cloud_errors",pd.DataFrame([buy]))
    iteration_data = {"date":datetime.now(),
                        "retrack_days" : retrack_days
                        ,"req" : req
                        ,"signal" : signal
                        ,"value" : value
                        ,"conservative" : conservative
                        ,"entry_strategy" : entry_strategy
                        ,"exit_strategy" : exit_strategy
                        ,"fee" : fee
                        ,"minimum_rows" : minimum_rows
                        ,"live" : live
                        ,"sleep_time" : sleep_time,
                        "status":status}
#         comet.store("cloud_iterrations",pd.DataFrame([iteration_data]))
except Exception as e:
    error_log = {"date":datetime.now(),"message":str(e)}
#         comet.store("cloud_errors",pd.DataFrame([error_log]))
comet.disconnect()

In [11]:
merged

,timestamp,low,high,open,close,volume,date,crypto,signal,velocity,inflection,p_sign_change,trade_id,price,size,time,bid,ask
0,1642291200,43016.040000,43128.060000,43098.200000,43122.000000,1.580360e+02,2022-01-15,BTC,0.012690,-1.677887,-0.502536,False,264521553,43121.010000,0.00020802,2022-01-16T00:32:38.477253Z,43121.000000,43121.010000
1,1642291200,1.286700,1.297000,1.292300,1.294700,1.275305e+06,2022-01-15,ADA,0.050211,-4.169563,2.536884,True,53987344,1.295500,384.03,2022-01-16T00:32:37.793447Z,1.295400,1.295500
2,1642291200,0.182500,0.184700,0.184500,0.184500,7.886297e+06,2022-01-15,DOGE,0.074549,-0.437434,0.417138,True,25493666,0.184600,150.9,2022-01-16T00:32:36.953764Z,0.184500,0.184600
3,1642291200,3317.880000,3336.060000,3327.810000,3336.050000,1.159625e+03,2022-01-15,ETH,0.029048,-3.152842,0.936521,True,207181899,3336.050000,0.20932498,2022-01-16T00:32:38.946658Z,3336.040000,3336.050000
4,1642291200,0.000031,0.000031,0.000031,0.000031,4.286402e+10,2022-01-15,SHIB,0.043919,-3.758108,2.276389,True,59858007,0.000031,551053,2022-01-16T00:32:38.694568Z,0.000031,0.000031
5,1642291200,86.880000,87.730000,87.290000,87.350000,8.353296e+03,2022-01-15,WLUNA,0.113732,0.738444,-2.840664,True,2980301,87.200000,2.82,2022-01-16T00:32:39.405526Z,87.200000,87.250000
6,1642291200,92.670000,93.270000,93.080000,93.130000,9.999353e+03,2022-01-15,AVAX,0.037545,-2.226200,-0.522311,False,9465134,93.140000,0.5,2022-01-16T00:32:34.140516Z,93.120000,93.140000
7,1642291200,147.000000,147.890000,147.750000,147.720000,2.651343e+03,2022-01-15,LTC,0.079667,0.981483,-2.641178,True,86174866,147.710000,0.20100537,2022-01-16T00:32:40.110419Z,147.670000,147.710000
8,1642291200,27.580000,27.990000,27.620000,27.990000,2.540227e+04,2022-01-15,DOT,0.084043,6.926160,-8.888865,True,13538371,27.990000,57.37,2022-01-16T00:32:38.046616Z,27.990000,28.000000
9,1642291200,2.340000,2.355100,2.346600,2.350000,5.006938e+05,2022-01-15,MATIC,0.037573,-2.736195,-3.620515,False,38696853,2.350000,0.4,2022-01-16T00:32:37.142705Z,2.349600,2.350100
